In [2]:
!pip install instalooter

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13724 sha256=f86d0314ddca9f2a284fe1f6c8bb791a13f5c1b980de156ad4c60041c86d515c
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1


In [10]:
import tensorflow as tf
import numpy as np

import cv2
import os
import math
import scipy as sp
import PIL

# Tensorflow
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2
#####from keras_tuner.tuners import RandomSearch

import scikitplot as skplt
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [19]:
#!pip install instalooter
import instalooter
from instalooter.looters import ProfileLooter, HashtagLooter
looter = HashtagLooter("고양이")
looter.download("./Cat/", media_count=20)
looter = HashtagLooter("강아지")
looter.download("./Dog/", media_count=20)

20

In [6]:
print(looter)

In [7]:
import tensorflow as tf

In [28]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
# 특정 샘플이 에러가 있다하더라도, 많은 batch_size의 샘플때문에 옳은 방향으로 학습이 됨
batch_size = 32

# Training 수
# epoch 한번이 전체를 한번 학습 (데이터가 너무 많은경우에는 epoch_step을 설정해야함)
epochs = 50

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
# 처음에는 0.001, callback 함수에 의해서 작아질 예정
Decay = 1e-6

img_width = 1080
img_height = 1080 #아주 큰이미지 (512 * 512를 썼을때도 있음)

In [47]:
# 예시 # Model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_tensor=None, 
#                                            input_shape=(img_width,img_height,3), pooling=None)
model = tf.keras.applications.ResNet50(include_top=False, input_shape=(img_width,img_height,3))

In [57]:
model.add(tf.keras.layers.Dense(2, activation="softmax"))

AttributeError: 'Functional' object has no attribute 'add'

In [48]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 1080, 1080,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 1086, 1086,   0           ['input_4[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, 540, 540, 64  9472        ['conv1_pad[0][0]']              
                                )                                                          

                                                                                                  
 conv2_block2_add (Add)         (None, 270, 270, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, 270, 270, 25  0           ['conv2_block2_add[0][0]']       
                                6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 270, 270, 64  16448       ['conv2_block2_out[0][0]']       
                                )                                                                 
                                                                                                  
 conv2_blo

                                8)                                                                
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 135, 135, 12  512        ['conv3_block2_2_conv[0][0]']    
 ization)                       8)                                                                
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 135, 135, 12  0          ['conv3_block2_2_bn[0][0]']      
 n)                             8)                                                                
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 135, 135, 51  66048       ['conv3_block2_2_relu[0][0]']    
                                2)                                                                
          

 ization)                                                                                         
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, 68, 68, 256)  0          ['conv4_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block1_2_conv (Conv2D)   (None, 68, 68, 256)  590080      ['conv4_block1_1_relu[0][0]']    
                                                                                                  
 conv4_block1_2_bn (BatchNormal  (None, 68, 68, 256)  1024       ['conv4_block1_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_blo

                                                                                                  
 conv4_block3_out (Activation)  (None, 68, 68, 1024  0           ['conv4_block3_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block4_1_conv (Conv2D)   (None, 68, 68, 256)  262400      ['conv4_block3_out[0][0]']       
                                                                                                  
 conv4_block4_1_bn (BatchNormal  (None, 68, 68, 256)  1024       ['conv4_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block4_1_relu (Activatio  (None, 68, 68, 256)  0          ['conv4_block4_1_bn[0][0]']      
 n)       

                                )                                 'conv4_block6_3_bn[0][0]']      
                                                                                                  
 conv4_block6_out (Activation)  (None, 68, 68, 1024  0           ['conv4_block6_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv5_block1_1_conv (Conv2D)   (None, 34, 34, 512)  524800      ['conv4_block6_out[0][0]']       
                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 34, 34, 512)  2048       ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_blo

 conv5_block3_3_conv (Conv2D)   (None, 34, 34, 2048  1050624     ['conv5_block3_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv5_block3_3_bn (BatchNormal  (None, 34, 34, 2048  8192       ['conv5_block3_3_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 conv5_block3_add (Add)         (None, 34, 34, 2048  0           ['conv5_block2_out[0][0]',       
                                )                                 'conv5_block3_3_bn[0][0]']      
                                                                                                  
 conv5_block3_out (Activation)  (None, 34, 34, 2048  0           ['conv5_block3_add[0][0]']       
          

In [49]:
# 제너레이터
# Online-augmentation 적용 Generator
DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    featurewise_center=True, #online augmentation
    featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Online-augmentation 비적용 Generator (Test용)
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")
# augmentaiton 하지 않음, test

In [16]:
pwd

'C:\\Users\\user\\이정훈강사님수업'

In [50]:
# Generator의 instance 생성 (Train)
TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    'C:/Users/user/이정훈강사님수업/catdog_train',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical",
    subset = "training")

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    'C:/Users/user/이정훈강사님수업/catdog_train',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode="categorical",
    subset = "validation")

# Generator의 instance 생성 (Test)
TEST_GENERATOR = DATAGEN_TEST.flow_from_directory(
    'C:/Users/user/이정훈강사님수업/catdog_test',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode='categorical')

Found 90 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [51]:
# Call-back 함수

# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='./catdog_model'+'-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}.hdf5',
            monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# 학습과정 진행사항 확인
TB = TensorBoard(log_dir='./catdog_tensorboard', write_graph=True, write_images=True)

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)
# patience(3) 번 동안 validation set에서의 loss가 떨어지지 않으면, learning rate를 0.8만큼 곱해라
# 0.001 -> 0.0008 -> 0.00064 -> 0.00051xx...

CALLBACK = [CP, TB, LR]
# fit_generator에 넣을 것

DeepLearning.compile(optimizer=
                     SGD(lr=LearningRate,decay=Decay, momentum=0.9, nesterov=True), 
                     loss='categorical_crossentropy',
                     metrics=['acc']

In [52]:
model.compile(SGD(lr=0.001,decay=1e-6, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy',
              metrics=['acc'])

C:\Users\user\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [38]:
tf.__version__

'2.8.0'

model.fit(train_data, 
          batch_size=128
          epochs=NUM_EPOCHS, 
          validation_data=(validation_inputs,validation_targets))

In [55]:
# 학습 시작
########## Training Start
model.fit(
        TRAIN_GENERATOR,
        batch_size=30,
        
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        #steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        
        epochs=5, #epochs 학습 오래하기 어려운 강의 상황이라 1로 바꿈
        callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=VALID_GENERATOR)
###########

C:\Users\user\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\user\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5


ValueError: in user code:

    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users

In [ ]:
model.fit

In [ ]:
model.